# DNN Practical: Ag Detection by Muon Spectroscopy

In this notebook, we attempt to solve a real problem in physics using a fully connected DNN.

We have a set of spectra from Muon spectroscopy experiments, from which we would like to detect whether or not a certain element is present in a sample. In this notebook, we are going to train a neural network to detect the presence of Ag. Through this practice, we will encounter and overcome a pitfall in deep learning known as **class imbalance**. We will also explore **early stopping** and saving checkpoints from the best performing model.

## About the data

The data in this example is generated from simulated muon spectroscopy experiments. First the data was generated for each individual element by simulating the spectral emmission lines of that element. Then for the mixed coumpounds the different elemental spectra were mixed in proportion to how much of that element is present in the compound. 

## Google Cloud Storage Boilerplate

The following two cells have some boilerplate to mount the Google Cloud Storage bucket containing the data used for this notebook to your Google Colab file system. **Even you are not using Google Colab, please make sure you run these two cells.** 

To access the data from Google Colab, you need to:

1. Run the first cell;
2. Follow the link when prompted (you may be asked to log in with your Google account);
3. Copy the Google SDK token back into the prompt and press `Enter`;
4. Run the second cell and wait until the data folder appears.

If everything works correctly, a new folder called `sciml-workshop-data` should appear in the file browser on the left. Depending on the network speed, this may take one or two minutes. Ignore the warning "You do not appear to have access to project ...". If you are running the notebook locally or you have already connected to the bucket, these cells will have no side effects.

In [1]:
import os 
# import Pytorch libraries
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

# helpers
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# need some certainty in data processing
np.random.seed(0)
torch.manual_seed(0)

In [23]:
# variables passed to bash; do not change
project_id = 'sciml-workshop'
bucket_name = 'sciml-workshop-data'
colab_data_path = '/content/sciml-workshop-data/'

try:
    from google.colab import auth
    auth.authenticate_user()
    google_colab_env = 'true'
    data_path = colab_data_path
except:
    google_colab_env = 'false'
    ###################################################
    ######## specify your local data path here ########
    ###################################################
    with open('../local_data_path.txt', 'r') as f: 
        data_path = f.read().splitlines()[0]


print(f"Using Colab: {google_colab_env}, Data path: {data_path}, exist: {os.path.isdir(data_path)}")

Using Colab: false, Data path: /mnt/materials/SciML/sciml-workshop/, exist: False


# Mount the workshop data

In [18]:
%%bash -s {google_colab_env} {data_path} {bucket_name}

# running locally
if ! $1; then
    echo "Running notebook locally."
    #exit
fi

if ! command -v s3fs &> /dev/null
then
    echo "Unable to find s3fs. Installing ..."
    apt -qq update
    apt -qq install s3fs fuse
fi

echo "Mounting bucket $3 to $2"
s3fs $3 $2 -o allow_other,nonempty,use_path_request_style,no_check_certificate,public_bucket=1,ssl_verify_hostname=0,host=https://s3.echo.stfc.ac.uk,url=https://s3.echo.stfc.ac.uk

Running notebook locally.
Mounting bucket sciml-workshop-data to /mnt/materials/SciML/sciml-workshop/


s3fs: unable to access MOUNTPOINT /mnt/materials/SciML/sciml-workshop/: No such file or directory


CalledProcessError: Command 'b'\n# running locally\nif ! $1; then\n    echo "Running notebook locally."\n    #exit\nfi\n\nif ! command -v s3fs &> /dev/null\nthen\n    echo "Unable to find s3fs. Installing ..."\n    apt -qq update\n    apt -qq install s3fs fuse\nfi\n\necho "Mounting bucket $3 to $2"\ns3fs $3 $2 -o allow_other,nonempty,use_path_request_style,no_check_certificate,public_bucket=1,ssl_verify_hostname=0,host=https://s3.echo.stfc.ac.uk,url=https://s3.echo.stfc.ac.uk\n'' returned non-zero exit status 1.

### If there is error mounting the bucket, manually verify that the s3fs command is correct by running the command in a new terminal;
```bash
sudo mkdir /mnt/materials/SciML/sciml-workshop/
sudo chmod 
sudo s3fs sciml-workshop-data /mnt/materials/SciML/sciml-workshop/ -o allow_other,nonempty,use_path_request_style,\
no_check_certificate,public_bucket=1,ssl_verify_hostname=0,host=https://s3.echo.stfc.ac.uk,url=https://s3.echo.stfc.ac.uk
```
### Use umount option with -l to unmount the bucket
```bash
sudo umount -l /mnt/materials/SciML/sciml-workshop/

---

# The dataset

### Read raw data

The raw data, which include the constituent elements and the Muon spectra of the samples, are stored in the pickle file `muon/Ag_muon_data.pkl`. We load this file into a `pandas` dataframe and take a quick look.

In [24]:
# read data
df = pd.read_pickle(data_path + 'sciml-workshop/muon-data/ag-muon-data-tight.pkl')
#print dimensions
print('Number of samples in the dataset: %d' % len(df['Spectra']))
print('Length of spectra for each sample: %d' % len(df['Spectra'][0]))

# print the first few data
df.head(n=5)

Number of samples in the dataset: 138613
Length of spectra for each sample: 1000


,Elements,oh,c,Spectra
8877,"[Si, Fe, Sb, Bi]",0,"[0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 132.19327401887..."
88555,"[Sb, Fe, Bi]",0,"[0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 106.91719527298..."
111153,"[Si, Sb]",0,"[0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 108.04741040971..."
27279,"[Fe, Cu, Bi]",0,"[0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 106.84704457348..."
26419,"[Si, Fe]",0,"[0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.82180543319..."


In the above table, the `Elements` and the `Spectra` columns show respectively the elements and the spectra of the samples. There are 138,613 samples in the dataset, and each spectrum is a series of 1000 positive reals. 

To get a feel for the complexity of picking out signals with Ag in multinary samples, we can plot some random spectra for three representative cases: 

* no Ag
* pure Ag
* Ag-Si binary

Note that we are plotting only the first part of each spectrum. Change `[0:150]` to `[:]` to show the full spectra.

In [ ]:
# conditions to select data
conditions = [
# no Ag
('no Ag', np.where(['Ag' not in elements for elements in df['Elements']])[0]),
# pure Ag
('pure Ag', np.where([['Ag'] == elements for elements in df['Elements']])[0]),
# Ag-Si
('Ag-Si binary', np.where([['Ag', 'Si'] == elements for elements in df['Elements']])[0])
]

# plot
ncond = len(conditions)
nplot = 4 # number of plots per condition
fig, axs = plt.subplots(nplot, ncond, dpi=200, figsize=(ncond * 5, nplot * 2), sharex=True, sharey=True)
plt.subplots_adjust(wspace=.1, hspace=.2)
for icond, cond in enumerate(conditions):
    for iplot, idata in enumerate(np.random.choice(cond[1], nplot)):
        axs[iplot, icond].plot(df['Spectra'][idata][150:700], c='C%d' % icond)
        axs[iplot, icond].set_xlabel('Sample %d: %s' % (idata, cond[0]), c='C%d' % icond)
        axs[iplot, icond].set_ylim(0, 100)

### Extract training data

The input data for our network will be the `Spectra` column, and we can use the `to_list()` method to convert it to a numpy array. The output data for our network will be a binary-valued one-hot vector: **0 for no Ag** in the sample and **1 otherwise**. One-hot encoding can be achieved by a simple for-loop. Also, it is important to normalise each spectrum between 0 and 1.

In [ ]:
###### input ######
# convert the 'Spectra' column to numpy
train_x = np.array(df['Spectra'].to_list())
# normalise each spectrum to [0, 1]
train_x /= np.max(train_x, axis=1)[:, np.newaxis]

###### output ######
# one-hot encoding: whether Ag is in 'Elements'
train_y = np.array(['Ag' in elements for elements in df['Elements']]).astype(np.int32)[:, np.newaxis]

# print data shapes
print("Shape of input: %s" % str(train_x.shape))
print("Shape of output: %s" % str(train_y.shape))

---

# Ag-detection by DNN

## 1. Try out a network


### Build the network

Based on what we have learnt in [DNN_basics.ipynb](DNN_basics.ipynb), design a simple neural network with `Linear` layers to detect Ag in the spectra. In general, it is not a straightforward task to determine the number of hidden layers and the number of neurons in each layer, which usually involves some trial and error. In this case, our input size is 1000 and output size is 1, so we'd better add a small layer before it, such as one with size 64.


In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim=1000, output_dim=1):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, 64)
        self.input_relu = nn.ReLU()
        
        self.hidden_layer = nn.Linear(64, 16)
        self.hidden_relu = nn.ReLU()
        
        self.out_layer = nn.Linear(16, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        
        x = self.input_layer(x)
        x = self.input_relu(x)
        
        x = self.hidden_layer(x)
        x = self.hidden_relu(x)
        
        x = self.out_layer(x)
        x = self.sigmoid(x)
        
        return x

# create a instance of the model
model = Model()
# print model summary
print(model)

# test the forward pass is working using dummy input
print(model(torch.randn(1, 1000)).detach().item())

### Create dataloaders 
We will convert the Numpy based train data to Pytorch dataloader suitable for our trainer function. We will also need to create a subset of data for validation by using the *final* 20% of the dataset. 

In [ ]:
def create_dataloaders(train_x, train_y, batch_size=64, val_split=0.2):
    
    idx = int((1-val_split)*len(train_x))

    train_dataset = TensorDataset(
        torch.tensor(train_x[:idx], dtype=torch.float32), 
        torch.tensor(train_y[:idx], dtype=torch.float32)
        )
    test_dataset = TensorDataset(
        torch.tensor(train_x[idx:], dtype=torch.float32),
        torch.tensor(train_y[idx:], dtype=torch.float32)
        )

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
    
    print(f"Train samples:{len(train_dataloader.dataset)}, Test samples:{len(test_dataloader.dataset)}, Batch size:{batch_size}")
    
    return train_dataloader, test_dataloader

### Build optimizer and loss function

We can keep using `Adam` for the `optimizer`. For the `loss`, since we are fitting to a range between 0 and 1, we can choose binary crossentropy `BCELoss`.

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# define loss function
loss_fn = nn.BCELoss()
# create a optimizer with constant learning rate of 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
```
</p>
</details>

### Create our  trainer function

As in the [DNN_basics.ipynb](DNN_basics.ipynb), we will create a custom trainer function to simplify the training and testing our model. We will initiate our Pytorch model, optimizers and loss inside the trainer so that we only need to our dataset and number of epochs. 

In [ ]:
def binary_accuracy(pred, ground_truth):
    return torch.where(torch.abs(pred - ground_truth) < .1)[0].numel() / pred.numel()

def train(dataloader, model, loss_fn, optimizer, device):

    model.train()
    
    train_loss, train_accuracy = 0, 0
    for num_batches, (X, y) in enumerate(dataloader):

        # place tensors to device
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred_y = model(X)

        # compute loss, default mean
        loss = loss_fn(pred_y, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # compute test loss and accuracy with threshold 0.5
        train_loss += loss.detach().item()
        train_accuracy += binary_accuracy(pred_y, y)
        
    train_loss /= num_batches
    train_accuracy /= num_batches
    
    return train_loss, train_accuracy

def test(dataloader, model, loss_fn, device):
    # set the model to eval mode
    model.eval()
    
    test_loss, test_accuracy = 0, 0
    with torch.no_grad():
        for num_batches, (X, y) in enumerate(dataloader):
            
            # place tensors to device
            X, y = X.to(device), y.to(device)
            
            # inference/prediction
            pred_y = model(X)

            # compute test loss and accuracy
            test_loss += loss_fn(pred_y, y).item()
            test_accuracy += binary_accuracy(pred_y, y)
            
    test_loss /= num_batches
    test_accuracy /= num_batches

    return test_loss, test_accuracy

def trainer(model, train_x, train_y, batch_size, epochs, learning_rate, device):
    
    model = model.to(device)
    
    loss_fn = nn.BCELoss()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train_dataloader, test_dataloader = create_dataloaders(train_x, train_y, batch_size)

    # bucket to hold history
    h = dict(train_loss=[], test_loss=[], train_accuracy=[], test_accuracy=[])
    
    # start timer
    start = time.time()
    
    for t in range(epochs):
        print(f"\nEpoch {t+1}/{epochs}: ", end='')
        
        train_loss, train_accuracy = train(train_dataloader, model, loss_fn, optimizer, device)

        test_loss, test_accuracy = test(test_dataloader, model, loss_fn, device)
        
        print(f"train_loss: {train_loss:0.4f}, test_loss: {test_loss:0.4f}, train_accuracy: {(100*train_accuracy):0.2f}%, test_accuracy: {(100*test_accuracy):0.2f}%")

        h['train_loss'].append(train_loss)
        h['test_loss'].append(test_loss)
        h['train_accuracy'].append(train_accuracy)
        h['test_accuracy'].append(test_accuracy)

    print(f"Done in {time.time()-start:.3f}secs!")
    
    return h

### Train the model

Let us train for 10 epochs first.

In [ ]:
np.random.seed(0)
torch.manual_seed(0)

device = torch.device('cuda:0')

model = Model()
print(model)
    
training_history = trainer(model, train_x, train_y, batch_size=64, epochs=10, 
                              learning_rate=0.001, device=device)

### Plot training history

For convenience, we define a function to plot a training history:

In [ ]:
# a function to plot training history
def plot_history(history, figsize=(12, 4)):
    plt.close('all')
    plt.figure(dpi=100, figsize=figsize)
    plt.subplot(1, 2, 1)
    plt.plot(history['train_accuracy'], label='Train')
    plt.plot(history['test_accuracy'], label='Test')
    plt.legend()
    plt.title("Accuracy")
    
    # plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['test_loss'], label='Test')
    plt.legend()
    plt.title("Loss")
    plt.show()

Now, plot the training history of the current model. They will look bizarre at this stage, as explained in the forthcoming section.

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
# plot training history
plot_history(training_history)
```
    
</p>
</details>

In [ ]:
plot_history(training_history)

## 2. Class imbalance

In the above history plot, notice how the accuracy of the model converges to a high value very quickly (>90% at the end of the first epoch). Such an odd history indicates that something could be wrong within our dataset.

### Data distribution

Let us inspect the distribution of the data using `plt.hist(train_y)`, paying special attention to the validation part (the final 20%).

In [ ]:
# plot distribution of data
plt.figure(dpi=100)
plt.hist(train_y, label='Whole dataset')
plt.hist(train_y[-len(train_y)//5:], label='Validation subset')
plt.xticks([0, 1], ['0: no Ag', '1: with Ag'])
plt.xlabel('label')
plt.ylabel('number of data')
plt.legend()
plt.show()

The histograms show that our dataset is dominated by samples labelled 0 or "no Ag", which account for over 95% of the data. Thus, if the model simply learns to *guess* "no Ag" in every sample, it can achieve 95% accuracy without learning anything meaningful. This problem is known as **class imbalance**.

To avoid this, we must balance the classes. There are a number of strategies we can take:

* Upsample the minority class;
* Downsample the majority class;
* Change the performance metric.

The best available option for our problem is to downsample the majority class, which can be easily achieved with `numpy`:

In [ ]:
# find original indices of 0 ('no Ag') and 1 ('with Ag')
id_no_Ag = np.where(train_y == 0)[0]
id_with_Ag = np.where(train_y == 1)[0]

# downsample 'no Ag' to the number of 'with Ag' by np.random.choice
id_no_Ag_downsample = np.random.choice(id_no_Ag, len(id_with_Ag))

# concatenate 'with Ag' and downsampled 'no Ag'
id_downsample = np.concatenate((id_with_Ag, id_no_Ag_downsample))

# shuffle the indices because they are ordered after concatenation
np.random.shuffle(id_downsample)

# finally get the balanced data
train_x_balanced = train_x[id_downsample]
train_y_balanced = train_y[id_downsample]

Re-exam the histograms of the balanced dataset after downsampling the majority:

In [ ]:
# plot distribution of downsampled data
plt.figure(dpi=100)
plt.hist(train_y_balanced, label='Whole dataset')
plt.hist(train_y_balanced[-len(train_y_balanced)//5:], label='Validation subset')
plt.xticks([0, 1], ['0: no Ag', '1: with Ag'])
plt.xlabel('label')
plt.ylabel('number of data')
plt.legend()
plt.show()

### Re-train the model

Now we can re-train the model with the balanced dataset. Simply change `train_x` and `train_y` to `train_x_balanced` and `train_y_balanced` in `trainer()` and repeat all the steps in [1. Try out a network](#1.-Try-out-a-network). Note that, to avoid the influence of the initial model state (weights and biases) left by the previous training (such as the one trained with the imbalanced dataset), we have to first re-define the model and reset the `optimizer` before calling `train` step. A larger `epochs` can be used because we now have much fewer data. 


**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python

# re-train the model

training_history_balanced = trainer(train_x_balanced, train_y_balanced, batch_size=256, epochs=500, device=device)

# plot training history
plot_history(training_history_balanced)
```
    
</p>
</details>

In [ ]:
np.random.seed(10)
torch.manual_seed(10)

model = Model()
    
training_history_balanced = trainer(model, train_x_balanced, train_y_balanced, batch_size=512, epochs=100, 
                                    learning_rate=0.001, device=device)

In [ ]:
# plot training history
plot_history(training_history_balanced)

### Early stopping

The droput has brought the training and validation losses closer to one another. However we also see that there are large oscillations in the validation performance. This is realted to the rather small training and validation set sizes. How can we make sure that we recover the model with the best performance on validation?


We can create custom `EarlyStopping` class to track the validation loss over training epochs and save the model every time there is a new best validation loss. We can then also tell our `trainer` to cease if the validation loss has not improved for some steps or epochs; 

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
    
def trainer(...):
    
    ....

    early_stopping = EarlyStopping(patience=5)
    for t in range(epochs):
    
        ....
        
        early_stopping(test_loss, model)
        if early_stopping.early_stop:
            break
    
```
    
</p>
</details>

---

## Exercises

Build a DNN to detect the presence of all the elements. To do this, you may go through the following steps:

1. Find all the elements appearing in the dataset; the answer will be `['Zn', 'Sb', 'Si', 'Fe', 'Ag', 'Cu', 'Bi']`.
2. Balance the dataset: if one of the elements appears much less times than the others, it is better to ignore it. Doing everything correctly, you will find the number of samples containing each element as shown in the following table. Therefore, we may ignore Ag in this network.


|  Element | # Samples |
|---|---|
|Zn| 51174|  
|Sb| 51132|  
|Si| 50909| 
|Fe| 50764|
|Ag| 10000|
|Cu| 50945|
|Bi| 50784|
    
3. Do one-hot encoding for the element list `['Zn', 'Sb', 'Si', 'Fe', 'Cu', 'Bi']`; if a sample contains Fe and Sb, e.g., the one-hot vector for this sample will be `[0, 1, 0, 1, 0, 0]`.
4. Build and train a DNN (with an output size of 6) to detect the presence of the six elements.

If doing everything correctly, you will find that the overall accuracy is around 60%. However, the model is not garbage. If we evaluate the accuracy for each element, we will find that the accracy for some of elements is nearly 0 while for the others nearly 100%. This means the dataset is agnostic to these elements, which lower the overall accuracy, but the model can still be used to predict the other elements with  high accuracy.

**Suggested Answer** 

<details> <summary>Show / Hide</summary> 
<p>
    
```python
##################
###### data ######
##################
element_list = ['Zn', 'Sb', 'Si', 'Fe', 'Cu', 'Bi']
train_y = []
for element in element_list:
    train_y.append(np.array([element in elements for elements in df['Elements']]).astype(int))
train_y = np.transpose(np.array(train_y))

# print data shapes
print("Shape of input: %s" % str(train_x.shape))
print("Shape of output: %s" % str(train_y.shape))

# create model
model = Model(output_dim=6)

# train the model
training_history = trainer(model, train_x, train_y, batch_size=256, epochs=100, 
                           learning_rate=0.001, device=device)
# plot training history
plot_history(training_history)


#####################
###### predict ######
#####################
train_x_tensor = torch.from_numpy(train_x).float().to(device)
train_y_tensor = torch.from_numpy(train_y).float().to(device)

model.eval()
with torch.no_grad():
    pred_y = model(train_x_tensor)

# show overall accuracy and accuracy for each element
print(f'Overall accuracy = {binary_accuracy(pred_y, train_y_tensor)}')

avg_acc = []
for i, element in enumerate(element_list):
    print(f'Accuracy for {element} = {binary_accuracy(pred_y[:, i:i+1], train_y_tensor[:, i:i+1])}') 
```
    
</p>
</details>